In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import re

In [ ]:
from timeseries_functions import index_to_datetime, weekly_resample, plot_all_df_columns, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

In [ ]:
from timeseries_functions import make_col_vector, make_design_matrix, fit_linear_trend,\
plot_trend_data, plot_linear_trend

In [ ]:
from AR_MA_functions import get_AR_model, plot_AR_model, get_AR_model_order_BIC,\
plot_BIC_AR_model, get_MA_model, plot_MA_model, get_MA_train_test_predictions,\
get_MA_train_test_MSE

In [ ]:
from ARIMA_functions import get_ARIMA_model, plot_ARIMA_model, plot_ARIMAX_model_save_fig, plot_ARIMA_resids,\
get_ARIMA_forecast, plot_ARIMA_forecast_and_CI, plot_data_plus_ARIMA_predictions, \
test_rolling_ARIMA_forecast,get_predictions_df_and_plot_rolling_ARIMA_forecast, get_ARIMAX_predictions

### import data

In [ ]:
# appointments = pd.read_csv('Appointments.csv')
appointments = pd.read_csv('./data/AppointmentsSince2015.csv')

In [ ]:
appointments.shape

In [ ]:
calls = pd.read_csv('./data/CallsRingCentral.csv')

In [ ]:
reason_for_visit = pd.read_csv('./data/MeetingReasonForVisits.csv')

In [ ]:
meeting_status = pd.read_csv('./data/MeetingStatus.csv')

In [ ]:
offices = pd.read_csv('./data/Offices.csv')

In [ ]:
providers_schedules = pd.read_csv('./data/ProvidersSchedulesLastest.csv')

### combine/merge dataframes

In [ ]:
doctors = ['Psychiatry', 'Child & Adolescent Psychiatry', ]
RN_PAs = ['Medical', 'Psych/Mental Health, Child & Adolescent', 'Psych/Mental Health', 'Physician Assistant']
therapists = ['Marriage & Family Therapist', 'Psychologist', 'Specialist/Technologist, Other', 'Clinical' ]

In [ ]:
appointments['Specialty'].loc[appointments['Specialty'].isin(doctors)]= 'doctor'
appointments['Specialty'].loc[appointments['Specialty'].isin(RN_PAs)] = 'RN/PA'
appointments['Specialty'].loc[appointments['Specialty'].isin(therapists)] = 'therapist'

In [ ]:
merged1 = pd.merge(left=appointments, right=reason_for_visit, how='left', left_on='MeetingReasonForVisitId',\
                  right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'MeetingReasonForVisitId': 'ReasonForVisitId', 'Name':'ReasonForVisitName', 'Description':'ReasonForVisitDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# merge in office name from offices df
merged1 = pd.merge(left=merged1, right=offices, how='left', left_on='OfficeId', right_on='id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'OfficeName', 'id_x':'id'})

In [ ]:
merged1.drop('id_y', axis=1, inplace=True)

In [ ]:
merged1 = pd.merge(left=merged1, right=meeting_status, how='left', left_on='MeetingStatusId', right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'MeetingStatusName', 'Description':'MeetingStatusDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# reorder columns within the df
ordered_columns = ['id', 'Patient', 'PatientAgeMeetingDate', 'PatientGender',
       'PatientState', 'PatientCity', 'PatientInsurance', 'Provider',
       'Specialty', 'AppointmentDate', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName',
       'MeetingStatusDescription', 'OfficeId',  'OfficeName', 'CreatedBy']

In [ ]:
merged1 = merged1[ordered_columns]

### Data Cleaning: 
#### filling NaN values

In [ ]:
# filling NaN values in Specialty
implied_therapy = ['Therapy', 'New Patient Therapy', ]
implied_doctor = ['Therapy Telepsychiatry','Follow up Telepsychiatry', 'New Patient Therapy Telepsychiatry',\
                  'New Patient MD Adult', 'New Patient MD Adult Telepsychiatry']
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_therapy)] = 'therapist'
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_doctor)] = 'doctor'

In [ ]:
# # convert date columns to datetime 
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')

In [ ]:
# calculate time between AppointmentCreated and AppointmentDate
merged1['DaysFromAppointmentCreatedToVisit'] = (merged1['AppointmentDate'] - merged1['AppointmentCreated']).dt.days

In [ ]:
merged1 = merged1.set_index('AppointmentDate')

In [ ]:
merged1.to_csv('./data/appointments_full.csv')

In [ ]:
merged2 = merged1.copy()

In [ ]:
# drop rows with missing specialty
merged2.dropna(subset=['Specialty'], how='all', inplace=True)

### Use the number of hours per day per provider for each specialty

In [ ]:
merged2['Hours_Spent'] = merged2['AppointmentDuration'] /60

In [ ]:
merged2['AppointmentDate'] = pd.to_datetime(merged2.index,format='%Y-%m-%d')

In [ ]:
merged2.shape
#merged2.columns, merged2.index

In [ ]:
# drop outlier appointments that are > 90 minutes long
merged2 = merged2[merged2['AppointmentDuration'] <= 90]

In [ ]:
merged2.index

In [ ]:
# change appointmentduration to hours
merged2['AppointmentDuration'] = merged2['AppointmentDuration'] / 60.0

In [ ]:
# group by specialty
doctors = merged2[merged2['Specialty'] == 'doctor']
RNPA = merged2[merged2['Specialty'] == 'RN/PA']
therapists = merged2[merged2['Specialty'] == 'therapist']

In [ ]:
doctors.columns

In [ ]:
columns = ['id', 'Patient', 'Provider', 'Specialty', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName', 'ReasonForVisitDescription']

In [ ]:
# keep only some columns
doctors = doctors[columns]
RNPA = RNPA[columns]
therapists = therapists[columns]

### doctors category

In [ ]:
doctors.head()

In [ ]:
doctors.index

In [ ]:
pd.value_counts(doctors['AppointmentDuration'])

#### separate new from existing patients

In [ ]:
# doctors
dr_new = doctors[doctors['ReasonForVisitDescription'].str.contains('New')]
dr_existing = doctors[~doctors['ReasonForVisitDescription'].str.contains('New')]

In [ ]:
pd.value_counts(dr_new['ReasonForVisitName'])

In [ ]:
pd.value_counts(dr_existing['ReasonForVisitName'])

In [ ]:
# drop columns where reason for visit name = Follow up, THerapy, returning patient existing patient dataframee
keep_cols = ['Follow up Telepsychiatry', 'Follow up', 'Therapy Telepsychiatry', 'Returning Patient', 'Returning Patient MD Adult']
dr_existing = dr_existing[dr_existing['ReasonForVisitName'].isin(keep_cols)]

In [ ]:
pd.value_counts(dr_existing['ReasonForVisitName'])

In [ ]:
dr_existing.columns

In [ ]:
# Get dr appointment hours
dr_new_hours = weekly_resample(dr_new['AppointmentDuration'])
dr_existing_hours = weekly_resample(dr_existing['AppointmentDuration'])

In [ ]:
dr_new_hours.index

In [ ]:
# keep data through April of this year
dr_new_hours = dr_new_hours[:'2018-04-30']
dr_existing_hours = dr_existing_hours[:'2018-04-30']

In [ ]:
dr_new_hours
# dr_existing_hours

In [ ]:
# drop partial first week from dr_existing_hours
dr_existing_hours = dr_existing_hours[1:]

In [ ]:
# dr_existing_hours

#### get number of unique providers seeing new and existing patients to use as exogenous variable

In [ ]:
# get dr number of unique providers for each dataframe
num_dr_new = dr_new['Provider'].resample('W-MON', lambda x: x.nunique())
num_dr_existing = dr_existing['Provider'].resample('W-MON', lambda x: x.nunique())

In [ ]:
# keep data through April of this year
num_dr_new = num_dr_new[:'2018-04-30']
# keep data through April of this year
num_dr_existing = num_dr_existing[:'2018-04-30']

In [ ]:
# drop partial first week
num_dr_existing = num_dr_existing[1:]

In [ ]:
# num_dr_new
# num_dr_existing

In [ ]:
num_dr_new.index

#### test for stationarity

In [ ]:
# dr hours with new patients
run_augmented_Dickey_Fuller_test(series=dr_new_hours, num_diffs=2)

In [ ]:
# dr hours with existing patients
run_augmented_Dickey_Fuller_test(series=dr_existing_hours, num_diffs=2)

#### determine number of AR and MA terms to add for each time series

In [ ]:
# Plot PACF of first difference of each series
params = {'figure.figsize': [4,3],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=dr_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of dr hours new patients'))
plot_partial_autocorrelation(series=dr_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of dr hours existing patients'))

In [ ]:
# Plot ACF of first difference of each series
plot_autocorrelation(series=dr_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of dr hours new patients'))
plot_autocorrelation(series=dr_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of dr hours existing patients'))

#### plot ARIMAX model for each Dr dataframe and save figure

In [ ]:
start_date = '2015-01-19' # b/c first row of data was differenced away
end_date = '2018-04-30'
end_pred = '2018-09-30'

In [ ]:
num_dr_new.index

In [ ]:
dr_new_hours.values[1].dtype

In [ ]:
def plot_ARIMAX_model_save_fig(data, order, exog_var, start, end, title='', xlabel='', ylabel='',
                              figname='figure.png'):
    """Plots ARIMAX model
    data: pandas Series
    order: (p,d,q) format
    exog_var = exogenous variable as pandas Series
    start/end: starting/ending dates for plot (x_axis)
    """
    results = ARIMA(endog=data, order=order, exog=exog_var).fit()
    fig = results.plot_predict(start=start, end=end,exog=exog_var)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()
    fig.savefig(figname)

In [ ]:
# new patients
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=dr_new_hours, order=(2,1,2), exog_var=num_dr_new, start=start_date,\
                end=end_pred, title='Dr new patients AR2/MA2', xlabel='Time', ylabel='Appointment Hours',\
                              figname='./images/dr_new_patients_AR[2]IMA[2]X.png')

In [ ]:
# existing patients
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=dr_existing_hours, order=(4,1,3), exog_var=num_dr_existing, start=start_date,\
                end=end_pred, title='Dr existing patients AR4/MA3', xlabel='Time', ylabel='Appointment Hours',
                              figname='./images/dr_existing_patients_AR[4]IMA[3]X.png')

#### get ARIMAX predictions for each Dr dataframe

In [ ]:
# get ARIMAX predictions and data+predictions dataframe for each Dr dataframe
# new patients
dr_new_forecast, dr_new_data_plus_forecast = get_ARIMAX_predictions(data=dr_new_hours, order=(2,1,2), start=start_date,\
                    end=end_pred, exog=num_dr_new, typ='levels')

In [ ]:
# existing patients
dr_existing_forecast, dr_existing_data_plus_forecast = get_ARIMAX_predictions(data=dr_existing_hours, order=(4,1,3), start=start_date,\
                    end=end_pred, exog=num_dr_existing, typ='levels')

In [ ]:
def get_ARIMAX_output_df(forecast_df, number_providers):
    # rename columns
    forecast_df.columns = ['Appointment_Hours', 'Predicted_Hours']
    # round hours columns
    forecast_df['Appointment_Hours'] = round(forecast_df['Appointment_Hours'], 2)
    forecast_df['Predicted_Hours'] = round(forecast_df['Predicted_Hours'], 2)
    # add number providers columns
    forecast_df['Number_Providers'] = number_providers
    # calculate hours per provider
    forecast_df['Hours_per_Provider'] = round(forecast_df['Appointment_Hours'] / forecast_df['Number_Providers'],2)
    # get average hours per provider
    avg_hours = forecast_df['Hours_per_Provider'].mean()
    # get predicted number of providers, rounded 
    forecast_df['Predicted_Num_Providers'] = round(forecast_df['Predicted_Hours'] / avg_hours, 1)
    return forecast_df

In [ ]:
# get full predictions dataframes
dr_new_data_plus_forecast = get_ARIMAX_output_df(dr_new_data_plus_forecast, num_dr_new)
dr_existing_data_plus_forecast = get_ARIMAX_output_df(dr_existing_data_plus_forecast, num_dr_existing)

In [ ]:
# output full forecast dataframes to csv
dr_new_data_plus_forecast.to_csv('./data/dr_new_patients_arimax_forecast.csv')
dr_existing_data_plus_forecast.to_csv('./data/dr_existing_patients_arimax_forecast.csv')

In [ ]:
# dr_new_data_plus_forecast
dr_existing_data_plus_forecast

In [ ]:
# get 8-16 week forecast new patients
dr_new_8_to_16wk_arimax = dr_new_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
dr_new_8_to_16wk_arimax.index = dr_new_8_to_16wk_arimax.index.date

In [ ]:
dr_new_8_to_16wk_arimax

In [ ]:
# get 8-16 week forecast existing patients
dr_existing_8_to_16wk_arimax = dr_existing_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
dr_existing_8_to_16wk_arimax.index = dr_existing_8_to_16wk_arimax.index.date

### RN/PAs

In [ ]:
RNPA = merged2[merged2['Specialty'] == 'RN/PA']

In [ ]:
# get new and existing patient dataframes
RNPA_new = RNPA[RNPA['ReasonForVisitDescription'].str.contains('New')]
RNPA_existing = RNPA[~RNPA['ReasonForVisitDescription'].str.contains('New')]

In [ ]:
# check value counts
# pd.value_counts(RNPA_new['ReasonForVisitName'])
pd.value_counts(RNPA_existing['ReasonForVisitName'])

In [ ]:
# drop columns where reason for visit name = Follow up, returning patient
# existing patient dataframe
keep_RNPA_cols = ['follow up Telepsyche', 'Follow up', 'follow', 'Returning Patient']
RNPA_existing = RNPA_existing[RNPA_existing['ReasonForVisitName'].isin(keep_RNPA_cols)]
pd.value_counts(RNPA_existing['ReasonForVisitName'])

In [ ]:
# Get RNPA appointment hours
RNPA_new_hours = weekly_resample(RNPA_new['AppointmentDuration'])
RNPA_existing_hours = weekly_resample(RNPA_existing['AppointmentDuration'])
# keep data through April of this year
RNPA_new_hours = RNPA_new_hours[:'2018-04-30']
RNPA_existing_hours = RNPA_existing_hours[:'2018-04-30']

In [ ]:
# RNPA_new_hours
RNPA_existing_hours

In [ ]:
# drop partial first week from RNPA_existing
RNPA_existing_hours = RNPA_existing_hours[1:]

In [ ]:
# get RNPA number of unique providers for each dataframe
num_RNPA_new = RNPA_new['Provider'].resample('W-MON', lambda x: x.nunique())
num_RNPA_existing = RNPA_existing['Provider'].resample('W-MON', lambda x: x.nunique())
# keep data through April of this year
num_RNPA_new = num_RNPA_new[:'2018-04-30']
# keep data through April of this year
num_RNPA_existing = num_RNPA_existing[:'2018-04-30']
# drop partial first week
num_RNPA_existing = num_RNPA_existing[1:]

In [ ]:
# test for stationarity of hours data
# RNPA hours with new patients
run_augmented_Dickey_Fuller_test(series=RNPA_new_hours, num_diffs=2)

In [ ]:
# RNPA hours with existing patients
run_augmented_Dickey_Fuller_test(series=RNPA_existing_hours, num_diffs=2)

In [ ]:
# Plot PACF of first difference of each series
params = {'figure.figsize': [4,3],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}

plot_partial_autocorrelation(series=RNPA_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of RNPA hours new patients'))
plot_partial_autocorrelation(series=RNPA_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of RNPA hours existing patients'))

In [ ]:
# Plot ACF of first difference of each series
plot_autocorrelation(series=RNPA_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of RNPA hours new patients'))
plot_autocorrelation(series=RNPA_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of RNPA hours new patients'))

In [ ]:
num_RNPA_new.index

In [ ]:
start_date = '2015-09-07' # b/c no new patients recorded until 8/31/15 and then had to take first difference
end_date = '2018-04-30'
end_pred = '2018-09-30'
# new patients
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=RNPA_new_hours, order=(3,1,1), exog_var=num_RNPA_new, start=start_date,\
                end=end_pred, title='RNPA new patients AR3/MA1', xlabel='Time', ylabel='Appointment Hours',\
                              figname='./images/RNPA_new_patients_AR[3]IMA[1]X.png')

In [ ]:
start_date = '2015-09-07' # b/c no new patients recorded until 8/31/15 and then had to take first difference
end_date = '2018-04-30'
end_pred = '2018-09-30'
# new patients
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=RNPA_existing_hours, order=(4,1,2), exog_var=num_RNPA_existing, start=start_date,\
                end=end_pred, title='RNPA existing patients AR4/MA2', xlabel='Time', ylabel='Appointment Hours',\
                              figname='./images/RNPA_existing_patients_AR[4]IMA[2]X.png')

In [ ]:
# get ARIMAX predictions and data+predictions dataframe for each dataframe
# new patients
RNPA_new_forecast, RNPA_new_data_plus_forecast = get_ARIMAX_predictions(data=RNPA_new_hours, order=(3,1,1), start=start_date,\
                    end=end_pred, exog=num_RNPA_new, typ='levels')
# get full predictions dataframe
RNPA_new_data_plus_forecast = get_ARIMAX_output_df(RNPA_new_data_plus_forecast, num_dr_new)

In [ ]:
# existing patients
RNPA_existing_forecast, RNPA_existing_data_plus_forecast = get_ARIMAX_predictions(data=RNPA_existing_hours, order=(4,1,2), start=start_date,\
                    end=end_pred, exog=num_RNPA_existing, typ='levels')
# get full predictions dataframe
RNPA_existing_data_plus_forecast = get_ARIMAX_output_df(RNPA_existing_data_plus_forecast, num_RNPA_existing)

In [ ]:
# get 8-16 week forecast new patients
RNPA_new_8_to_16wk_arimax = RNPA_new_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
RNPA_new_8_to_16wk_arimax.index = RNPA_new_8_to_16wk_arimax.index.date

In [ ]:
# get 8-16 week forecast existing patients
RNPA_existing_8_to_16wk_arimax = RNPA_existing_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
RNPA_existing_8_to_16wk_arimax.index = RNPA_existing_8_to_16wk_arimax.index.date

### Therapists

In [ ]:
therapists = merged2[merged2['Specialty'] == 'therapist']

In [ ]:
# # doctors
# dr_new = doctors[doctors['ReasonForVisitDescription'].str.contains('New')]
# dr_existing = doctors[~doctors['ReasonForVisitDescription'].str.contains('New')]
# pd.value_counts(dr_new['ReasonForVisitName'])
# pd.value_counts(dr_existing['ReasonForVisitName'])
# # drop columns where reason for visit name = Follow up, THerapy, returning patient existing patient dataframee
# keep_cols = ['Follow up Telepsychiatry', 'Follow up', 'Therapy Telepsychiatry', 'Returning Patient', 'Returning Patient MD Adult']
# dr_existing = dr_existing[dr_existing['ReasonForVisitName'].isin(keep_cols)]
# pd.value_counts(dr_existing['ReasonForVisitName'])
# # Get dr appointment hours
# dr_new_hours = weekly_resample(dr_new['AppointmentDuration'])
# dr_existing_hours = weekly_resample(dr_existing['AppointmentDuration'])
# # keep data through April of this year
# dr_new_hours = dr_new_hours[:'2018-04-30']
# dr_existing_hours = dr_existing_hours[:'2018-04-30']
# # drop partial first week from dr_existing_hours
# dr_existing_hours = dr_existing_hours[1:]
# # get dr number of unique providers for each dataframe
# num_dr_new = dr_new['Provider'].resample('W-MON', lambda x: x.nunique())
# num_dr_existing = dr_existing['Provider'].resample('W-MON', lambda x: x.nunique())
# # keep data through April of this year
# num_dr_new = num_dr_new[:'2018-04-30']
# # keep data through April of this year
# num_dr_existing = num_dr_existing[:'2018-04-30']
# # drop partial first week
# num_dr_existing = num_dr_existing[1:]
# # test for stationarity
# # dr hours with new patients
# run_augmented_Dickey_Fuller_test(series=dr_new_hours, num_diffs=2)
# # dr hours with existing patients
# run_augmented_Dickey_Fuller_test(series=dr_existing_hours, num_diffs=2)
# # Plot PACF of first difference of each series
# params = {'figure.figsize': [4,3],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
# 'lines.linewidth': 2}
# plot_partial_autocorrelation(series=dr_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of dr hours new patients'))
# plot_partial_autocorrelation(series=dr_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='PACF {}'.format('first difference of dr hours existing patients'))
# # Plot ACF of first difference of each series
# plot_autocorrelation(series=dr_new_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of dr hours new patients'))
# plot_autocorrelation(series=dr_existing_hours.diff()[1:], params=params, lags=30, alpha=0.05, title='ACF {}'.format('first difference of dr hours existing patients'))
# # dr_new_hours use: AR2/MA2
# # dr_existing_hours use: AR4/MA3
# start_date = '2015-01-19' # b/c first row of data was differenced away
# end_date = '2018-04-30'
# end_pred = '2018-09-30'
# # new patients
# params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
# 'lines.linewidth': 2}
# plt.rcParams.update(params)

# plot_ARIMAX_model_save_fig(data=dr_new_hours, order=(2,1,2), exog_var=num_dr_new, start=start_date,\
#                 end=end_pred, title='Dr new patients AR2/MA2', xlabel='Time', ylabel='Appointment Hours',\
#                               figname='./images/dr_new_patients_AR[2]IMA[2]X.png')
# # get ARIMAX predictions and data+predictions dataframe for each Dr dataframe
# # new patients
# dr_new_forecast, dr_new_data_plus_forecast = get_ARIMAX_predictions(data=dr_new_hours, order=(2,1,2), start=start_date,\
#                     end=end_pred, exog=num_dr_new, typ='levels')
# # get full predictions dataframes
# dr_new_data_plus_forecast = get_ARIMAX_output_df(dr_new_data_plus_forecast, num_dr_new)
# dr_existing_data_plus_forecast = get_ARIMAX_output_df(dr_existing_data_plus_forecast, num_dr_existing)
# # get 8-16 week forecast new patients
# dr_new_8_to_16wk_arimax = dr_new_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# # keep only date in index, drop time
# dr_new_8_to_16wk_arimax.index = dr_new_8_to_16wk_arimax.index.date

### number of patients vs number of doctors

#### look at ratio of providers to patients

In [ ]:
dr_num_patients = doctors['id'].resample('W-MON', lambda x: x.nunique())

In [ ]:
dr_num_providers = doctors['Provider'].resample('W-MON', lambda x: x.nunique())

In [ ]:
dr_num_patients.index

In [ ]:
patients_to_dr = dr_num_patients / dr_num_providers 

In [ ]:
dr_to_patient = (dr_num_providers / dr_num_patients) * 100

In [ ]:
dr_to_patient

In [ ]:
patients_to_dr

In [ ]:
# resample data to weekly
doctors = doctors['Provider'].resample('W-MON', lambda x: x.nunique())
RN_PA = RN_PA['Provider'].resample('W-MON', lambda x: x.nunique())
therapists = therapists['Provider'].resample('W-MON', lambda x: x.nunique())